## 토픽 모델링
* 실습을 위해 pyLDAvis 설치가 필요합니다. 
* colab사용시 설치 후에도 제대로 동작하지 않거나 오류가 나면 런타임을 재실행 해주세요.

In [ ]:
# !pip install -U -q pyLDAvis

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 라이브러리 로드

In [ ]:
# 필요 라이브러리를 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 로드

In [ ]:
# 수집한 데이터셋을 불러옵니다.
df = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df.shape

In [ ]:
# 결측치가 있다면 제거합니다.
df = df.dropna()
df.shape

In [ ]:
# 결측치를 확인합니다.
df.isnull().sum()

## 문서 만들기
* 제목과 내용을 함께 사용합니다.

In [ ]:
df["문서"] = df["제목"] + " " + df["내용"]

## 벡터화

* BOW(bag of words)
    * 가장 간단하지만 효과적이라 널리쓰이는 방법
    * 장, 문단, 문장, 서식과 같은 입력 텍스트의 구조를 제외하고 각 단어가 이 말뭉치에 얼마나 많이 나타나는지만 헤아립니다.
    * 구조와 상관없이 단어의 출현횟수만 세기 때문에 텍스트를 담는 가방(bag)으로 생각할 수 있습니다.
    * BOW는 단어의 순서가 완전히 무시 된다는 단점이 있다. 예를 들어 의미가 완전히 반대인 두 문장이 있다고 합니다.
        - `it's bad, not good at all.` 
        - `it's good, not bad at all.` 
    * 위 두 문장은 의미가 전혀 반대지만 완전히 동일하게 반환됩니다.
    * 이를 보완하기 위해 n-gram을 사용하는 데 BOW는 하나의 토큰을 사용하지만 n-gram은 n개의 토큰을 사용할 수 있도록 합니다.
    * min_df는 문서에 특정 단어가 최소 몇 번 이상 문서에 등장하는 단어를 가방에 담겠다는 의미입니다.

* [Bag-of-words model - Wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)


## CountVectorizer

단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩 벡터를 생성합니다.
1. 문서를 토큰 리스트로 변환한다.
2. 각 문서에서 토큰의 출현 빈도를 센다.
3. 각 문서를 BOW 인코딩 벡터로 변환한다.
4. 매개 변수
* analyzer : 단어, 문자 단위의 벡터화 방법 정의
* ngram_range : BOW 단위 수 (1, 3) 이라면 1개~3개까지 토큰을 묶어서 벡터화
* max_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 높은 용어(말뭉치 관련 불용어)는 제외 (기본값=1.0)
    * max_df = 0.90 : 문서의 90% 이상에 나타나는 단어 제외
    * max_df = 10 : 10개 이상의 문서에 나타나는 단어 제외
* min_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 낮은 용어는 제외합니다. 컷오프라고도 합니다.(기본값=1.0)
    * min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 제외
    * min_df = 10 : 문서에 10개 미만으로 나타나는 단어 제외
* stop_words : 불용어 정의
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
# 단어들의 카운트(출현 빈도(frequency))로 여러 문서들을 벡터화하기 위해 CountVectorizer를 불러옵니다.
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=["돋움", "경우", "또는"])

### 참고: fit, transform, fit_transfrom의 차이점
- fit(): 원시 문서에 있는 모든 토큰의 어휘 사전을 배웁니다.
- transform(): 문서를 문서 용어 매트릭스로 변환합니다. transform 이후엔 매트릭스로 변환되어 숫자형태로 변경됩니다.
- fit_transform(): 어휘 사전을 배우고 문서 용어 매트릭스를 반환합니다. fit 다음에 변환이 오는 것과 동일하지만 더 효율적으로 구현됩니다.

* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit_transform

In [ ]:
# fit_transform을 사용하여 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성합니다.
dtm_cv = cv.fit_transform(df["문서"])

In [ ]:
# cv.vocabulary_ 를 봅니다.
# cv.vocabulary_

In [ ]:
cv_cols = cv.get_feature_names()

In [ ]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 one-hot-vector를 확인합니다.
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인합니다.

pd.DataFrame(dtm_cv.toarray(), columns=cv_cols).sum().sort_values()

## BOW<sup>bag of word</sup> 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)


```
LDA는 이산 자료들에 대한 확률적 생성 모형이다. 문자 기반의 자료들에 대해 쓰일 수 있으며 사진 등의 다른 이산 자료들에 대해서도 쓰일 수 있다. 기존의 정보 검색분야에서 LDA와 유사하게 문헌 내의 잠재적인 의미구조를 파악하려는 시도들은 계속 이루어져 왔다. TF-IDF를 필두로 하여 잠재 의미 분석(Latent semantic indexing, LSI), 확률 잠재 의미 분석(Probabilistic latent semantic analysis, pLSA)등을 거쳐 LDA로 도달하게 되었고, 이는 토픽 모델링이라 불리는 분야를 탄생시켰다. 확률 잠재 의미 분석은 확률 잠재 의미 인덱싱(probabilistic latent semantic indexing, pLSI) 라고도 한다.

LDA에는 몇 가지 가정이 있는데 그 중 중요한 것은 단어의 교환성(exchangeability)이다. 이는 '단어 주머니(bag of words)'라고 표현하기도 한다. 교환성은 단어들의 순서는 상관하지 않고 오로지 단어들의 유무만이 중요하다는 가정이다. 예를 들어, 'Apple is red'와 'Red is apple' 간에 차이가 없다고 생각하는 것이다. 단어의 순서를 무시할 경우 문헌은 단순히 그 안에 포함되는 단어들의 빈도수만을 가지고 표현이 가능하게 된다. 이 가정을 기반으로 단어와 문서들의 교환성을 포함하는 혼합 모형을 제시한 것이 바로 LDA이다. 하지만 단순히 단어 하나를 단위로 생각하는 것이 아니라 특정 단어들의 묶음을 한 단위로 생각하는 방식(n-gram)으로 LDA의 교환성 가정을 확장시킬 수도 있다.

LDA는 문헌의 주제를 찾기 위한 방법으로 고안되었지만, 이미지, 소리 등 텍스트 처리 이외의 다양한 분야에 쓰일 수 있고 이산 자료들, 즉 불연속적인 자료들뿐만 아니라 연속적인 자료들에 대해서 적용 할 수 있고 또한 다항 분포가 아닌 자료들에 대해서도 적용 할 수 있는 가능성이 있다.
```
출처 : https://ko.wikipedia.org/wiki/%EC%9E%A0%EC%9E%AC_%EB%94%94%EB%A6%AC%ED%81%B4%EB%A0%88_%ED%95%A0%EB%8B%B9

문서에 대한 클러스터 연관성을 찾는 데 사용되는 확률론적 모델입니다.
두 가지 확률 값을 사용하여 문서를 군집화합니다.

- P(단어 | 주제): 특정 단어가 특정 주제와 연관될 확률. 이 첫 번째 확률 집합은 워드 X 주제 행렬로도 간주됩니다.
- P(주제 | 문서): 문서와 관련된 항목. 이 두 번째 확률 집합은 주제 X 문서 행렬로 간주됩니다.

확률 값은 모든 단어, 주제 및 문서에 대해 계산됩니다.

* API documentation: https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [ ]:
# 정답인 '분류'의 유일한 값을 확인하여 주제 수를 확인합니다.
df["분류"].value_counts()

In [ ]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 불러옵니다.
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정합니다.(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수입니다.(기본값=10)

from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 10
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, random_state=42)

In [ ]:
# LDA_model 에 dtm_cv 를 넣어 학습합니다.
LDA_model.fit(dtm_cv)

### pyLDAvis

* pyLDAvis는 파이썬의 토픽모델링을 구현시켜주는 좋은 툴입니다.
* 모델링에서 최적화 시킨 토픽별 토픽을 대표하는 단어들을 반환시킨 후 PCA를 통해 2차원에 mapping시킵니다.
* 왼쪽 2차원 버블 차트는 PCA에 의해 변환된 토픽들이며, 오른쪽 bar 차트는 해당 토픽을 대표하는 단어들로 구성되어 있습니다. 
* 해당 단어들은 relevance라는 measure에 의해 대표되며, relevance는 주제(Topic) 안에 있는 용어(Term)을 평가하는 기준입니다.

In [ ]:
# !pip install -U -q pyLDAvis

In [ ]:
# 토픽 모델링에 이용되는 LDA 모델의 학습 결과를 시각화하는 Python 라이브러리인 pyLDAvis를 불러옵니다.
# mds(Multi-Dimensional Scaling)는 데이터 포인트 간의 거리를 보존하면서 차원을 축소하는 기법입니다.
# t-SNE(t-Stochastic Neighbor Embedding)은 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용합니다.

import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model, dtm_cv, cv, mds='tsne')

## TF-IDF

TF-IDF(Term Frequency - Inverse Document Frequency)

정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다. 문서의 핵심어를 추출하거나, 검색 엔진에서 검색 결과의 순위를 결정하거나, 문서들 사이의 비슷한 정도를 구하는 등의 용도로 사용할 수 있다.

TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용 되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.

IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

* 출처 : [tf-idf - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/Tf-idf)



## TfidfVectorizer

TF-IDF 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어(낮은 구별력)의 경우 가중치를 축소하는 방법

매개변수
* norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정한다. 
    - L2 : 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이고 기본 값
    - L1 : 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절
* smooth_idf=False
    - 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정
* sublinear_tf=False
* use_idf=True
    - TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer


In [ ]:
# TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩하여 벡터화하기 위해 TfidfVectorizer를 불러옵니다.

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=["돋움", "경우", "또는", "있습니다", "있는", "합니다"])
tfidf

In [ ]:
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성합니다.
dtm_tfidf = tfidf.fit_transform(df["문서"])

In [ ]:
# tfidf.vocabulary_ 의 번호는 정렬 순으로 되어 있습니다.
# tfidf.vocabulary_
cols_tfidf = tfidf.get_feature_names()

In [ ]:
# dtm_tf를 axis=0(수직 방향으로) 기준으로 합계를 낸 dist 변수를 생성합니다.
# dist 변수를 vocabulary_ 순으로 정렬하여 비율을 확인합니다.
dist = np.sum(dtm_tfidf, axis=0)
pd.DataFrame(dist, columns=cols_tfidf).T.sort_values(by=0).tail(10)

In [ ]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 가중치를 적용한 vector를 확인합니다.
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인합니다.
pd.DataFrame(dtm_tfidf.toarray(), columns=cols_tfidf)

## TF-IDF 잠재 디리클레 할당(LDA)

In [ ]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 불러옵니다.
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정합니다.(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수입니다.(기본값=10)

NUM_TOPICS = 10 
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, 
                                      max_iter=30, 
                                      random_state=42)

In [ ]:
# dtm_tfidf 를 LDA_model로 학습시킵니다.
LDA_model.fit(dtm_tfidf)

In [ ]:
# 토픽 모델링에 이용되는 LDA 모델의 학습 결과를 시각화하는 Python 라이브러리인 pyLDAvis를 불러옵니다.
# mds(Multi-Dimensional Scaling)는 데이터 포인트 간의 거리를 보존하면서 차원을 축소하는 기법입니다.
# t-SNE(t-Stochastic Neighbor Embedding)은 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용합니다.

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model, dtm=dtm_tfidf, vectorizer=tfidf, mds='tsne')

## 코사인 유사도

* 각 단어를 축으로 하는 특성 공간(feature space)에서 문서들을 하나의 위치로 보고 
* 특성 공간 상에서 거리를 이용해 두 문서의 유사성(similarity)을 측정하는 방식으로 다음과 같은 유사도 측정 방법이 있음
    * 유클리드 거리(euclidean distance)
    * 코사인 유사도(cosine similarity)
    * 맨해튼 거리(Manhattan distance)
    * 자카드 유사도(Jaccard similarity)

**코사인 유사도**

- 코사인 유사도는 원점(모든 단어의 빈도가 0인 경우)에서 보았을 때 두 문서의 각도에 바탕을 둔 거리 측정 방식 
- 내적공간의 두 벡터간 각도의 코사인값을 이용하여 측정된 벡터간의 유사한 정도를 의미한다. 각도가 0°일 때의 코사인값은 1이며, 다른 모든 각도의 코사인값은 1보다 작다. 따라서 이 값은 벡터의 크기가 아닌 방향의 유사도를 판단하는 목적으로 사용되며, 두 벡터의 방향이 완전히 같을 경우 1, 90°의 각을 이룰 경우 0, 180°로 완전히 반대 방향인 경우 -1의 값을 갖는다. 이 때 벡터의 크기는 값에 아무런 영향을 미치지 않는다. 코사인 유사도는 특히 결과값이 [0,1]의 범위로 떨어지는 양수 공간에서 사용된다.

* 출처: https://ko.wikipedia.org/wiki/%EC%BD%94%EC%82%AC%EC%9D%B8_%EC%9C%A0%EC%82%AC%EB%8F%84
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
# df.head()

In [ ]:
# 등장 빈도에 기반하여, 코사인 유사도 알고리즘 적용해봅니다.
# 첫 행의 "아빠 육아 휴직 장려금"과 비슷한 데이터 정렬해봅니다.

from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(dtm_tfidf[0] , dtm_tfidf)
result_list = similarity_simple_pair.tolist()[0]

In [ ]:
# result_list를 "유사도" 파생변수로 생성하고 유사도가 높은 순으로 정렬합니다.

df["유사도"] = result_list
df[["분류", "제목", "유사도"]].sort_values(by="유사도", ascending=False).head(10)